In [ ]:
from typing import Optional, Literal
import random
import copy


In [ ]:
# Example 4.1
class GridWorld:
    def __init__(self):
        self.current_state = (1, 1)
        self.end_states = [(0, 0), (3, 3)]
        self.BOARD_SIZE = 4
        self.random_environment = 0

    def is_final(self, state):
        return state in self.end_states

    def available_actions(self):
        return ["up", "down", "left", "right"]

    def get_transitions(self, state, action):
        actions = self.available_actions()

        transitions = []
        intended_prob = 1
        other_prob = 0.0 / (len(actions) - 1)

        for a in actions:
            if a == action:
                prob = intended_prob
            else:
                prob = other_prob

            next_state, reward = self._deterministic_step(state, a)
            transitions.append((prob, next_state, reward))

        return transitions

    def _deterministic_step(self, state, action):
        if action == "up":
            next_state = (state[0] - 1, state[1])
        elif action == "down":
            next_state = (state[0] + 1, state[1])
        elif action == "left":
            next_state = (state[0], state[1] - 1)
        elif action == "right":
            next_state = (state[0], state[1] + 1)

        if not self.is_valid_state(next_state):
            next_state = state

        reward = -1

        return next_state, reward

    def is_valid_state(self, state):
        return 0 <= state[0] < 4 and 0 <= state[1] < 4
    
class FrozenLake:
    def __init__(self):
        self.current_state = (0, 0)
        self.end_states = [(3, 3)]
        self.traps = [(1, 1), (1, 3), (2, 3), (3, 0)]
        self.BOARD_SIZE = 4
        self.random_environment = 0

    def is_final(self, state):
        return state in self.end_states
    
    def is_trap(self, state):
        return state in self.traps

    def available_actions(self):
        return ["up", "down", "left", "right"]

    def get_transitions(self, state, action):
        actions = self.available_actions()

        transitions = []
        intended_prob = 0.7
        other_prob = 0.3 / (len(actions) - 1)

        for a in actions:
            if a == action:
                prob = intended_prob
            else:
                prob = other_prob

            next_state, reward = self._deterministic_step(state, a)
            transitions.append((prob, next_state, reward))

        return transitions

    def _deterministic_step(self, state, action):
        if action == "up":
            next_state = (state[0] - 1, state[1])
        elif action == "down":
            next_state = (state[0] + 1, state[1])
        elif action == "left":
            next_state = (state[0], state[1] - 1)
        elif action == "right":
            next_state = (state[0], state[1] + 1)

        if not self.is_valid_state(next_state):
            next_state = state

        if self.is_final(next_state):
            reward = 1
        elif self.is_trap(next_state):
            reward = -1
        else:
            reward = 0

        return next_state, reward

    def is_valid_state(self, state):
        return 0 <= state[0] < 4 and 0 <= state[1] < 4

In [ ]:
def policy_evaluation():
    grid = FrozenLake()
    discounted_factor = 1
    state_value = [[random.random() for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
    last_state_value = [[random.random() for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
    actions = grid.available_actions()
    eps = 1e-4
    action_prob = 1 / 4

    def difference_states():
        total_diff = 0
        for i in range(grid.BOARD_SIZE):
            for j in range(grid.BOARD_SIZE):
                total_diff += abs(state_value[i][j] - last_state_value[i][j])  # absolute difference
        return total_diff

    while difference_states() >= eps:
        state_value = copy.deepcopy(last_state_value)
        for i in range(grid.BOARD_SIZE):
            for j in range(grid.BOARD_SIZE):
                # Search in all states
                value = 0
                if grid.is_final((i, j)):
                    value = 0
                else:
                    for action in actions:
                        for prob, next_state, reward in grid.get_transitions((i,j), action):
                            value += action_prob * prob * (reward + discounted_factor * state_value[next_state[0]][next_state[1]])
                last_state_value[i][j] = value

    for i in range(grid.BOARD_SIZE):
        for j in range(grid.BOARD_SIZE):
            print(f"{state_value[i][j]:.2f}", end=" ")
        print()
        

policy_evaluation()

In [ ]:
def policy_iteration():
    grid = FrozenLake()
    discounted_factor = 0.9
    action_prob = 1 / 4

    def initialize_random_action():
        action = [[None for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
        for i in range(grid.BOARD_SIZE):
            for j in range(grid.BOARD_SIZE):
                if grid.is_final((i, j)):
                    continue
                action[i][j] = random.choice(grid.available_actions())
        return action
            

    actions = initialize_random_action()
    new_actions = initialize_random_action()
    
    
    
    # policy evaluation over deterministic policy
    def policy_evaluation():
        last_state_value = [[random.random() for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
        state_value = [[random.random() for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
        eps = 1e-14
        def difference_states():
            total_diff = 0
            for i in range(grid.BOARD_SIZE):
                for j in range(grid.BOARD_SIZE):
                    total_diff += abs(state_value[i][j] - last_state_value[i][j])  # absolute difference
            return total_diff

        while difference_states() >= eps:
            state_value = copy.deepcopy(last_state_value)
            for i in range(grid.BOARD_SIZE):
                for j in range(grid.BOARD_SIZE):
                    # Search in all states
                    value = 0
                    if grid.is_final((i, j)):
                        value = 0
                    else:
                        for prob, next_state, reward in grid.get_transitions((i,j), actions[i][j]):
                            value += prob * (reward + discounted_factor * state_value[next_state[0]][next_state[1]])
                    last_state_value[i][j] = value
        return state_value

    def policy_improvement(state_value):
        for i in range(grid.BOARD_SIZE):
            for j in range(grid.BOARD_SIZE):

                if grid.is_final((i, j)):
                    new_actions[i][j] = None
                    continue

                max_value = -float('inf')
                for action in grid.available_actions():
                    value = 0
                    for prob, next_state, reward in grid.get_transitions((i,j), action):
                        value += prob * (reward + discounted_factor * state_value[next_state[0]][next_state[1]])
                    if value > max_value:
                        max_value = value
                        new_actions[i][j] = action
        
    
    while not actions == new_actions:
        actions = copy.deepcopy(new_actions)
        state_value = policy_evaluation()
        policy_improvement(state_value)
        
        

    for i in range(grid.BOARD_SIZE):
        for j in range(grid.BOARD_SIZE):
            print(f"{state_value[i][j]:.2f}", end=" ")
        print()

    for i in range(grid.BOARD_SIZE):
        for j in range(grid.BOARD_SIZE):
            print(f"{actions[i][j]}", end=" ")
        print()
    
policy_iteration()


In [ ]:
def value_iteration():
    grid = FrozenLake()
    discounted_factor = .9
    state_value = [[random.random() for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
    last_state_value = [[random.random() for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
    actions = [[None for _ in range(grid.BOARD_SIZE)] for _ in range(grid.BOARD_SIZE)]
    eps = 1e-4

    def difference_states():
        total_diff = 0
        for i in range(grid.BOARD_SIZE):
            for j in range(grid.BOARD_SIZE):
                total_diff += abs(state_value[i][j] - last_state_value[i][j])  # absolute difference
        return total_diff
    
    while difference_states() >= eps:
        last_state_value = copy.deepcopy(state_value)

        for i in range(grid.BOARD_SIZE):
            for j in range(grid.BOARD_SIZE):

                if grid.is_final((i, j)) or grid.is_trap((i, j)):
                    state_value[i][j] = 0
                    continue

                max_value = -float('inf')
                for action in grid.available_actions():
                    value = 0
                    for prob, next_state, reward in grid.get_transitions((i,j), action):
                        if next_state == (i,j): reward =  -1
                        value += prob * (reward + discounted_factor * state_value[next_state[0]][next_state[1]])
                    
                    if value > max_value:
                        state_value[i][j] = value
                        actions[i][j] = action
                        max_value = value

    
    for i in range(grid.BOARD_SIZE):
        for j in range(grid.BOARD_SIZE):
            print(f"{state_value[i][j]:.2f}", end=" ")
        print()

    for i in range(grid.BOARD_SIZE):
        for j in range(grid.BOARD_SIZE):
            print(f"{actions[i][j]}", end=" ")
        print()
    
value_iteration()
                        